In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model

Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
engine.execute('select * from Measurement').fetchall()

# Exploratory Climate Analysis

In [ ]:
engine.execute('select * from Station').fetchall()

In [ ]:
# Earliest Date
session.query(Measurement.date).order_by(Measurement.date).first()

In [ ]:
# Latest Date
last_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]
last_date

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
# Calculate the date 1 year ago from the last data point in the database

date_yr_ago = dt.date(2017,8,23) - dt.timedelta(days=365)
date_yr_ago
# Perform a query to retrieve the data and precipitation scores
last_12_months =session.query(Measurement.date, Measurement.prcp).filter(Measurement.date > date_yr_ago).\
order_by(Measurement.date).all()
last_12_months[0]
# last_12_months


In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
# Sort the dataframe by date
prcp_scores = pd.DataFrame(last_12_months, columns=['Date', 'Precipitation'])

prcp_scores.set_index('Date')
prcp_scores =prcp_scores.sort_index(ascending = True)
prcp_scores


In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
ax = prcp_scores.plot('Date','Precipitation')
#ax= prcp_scores.plot.bar(figsize=(10,8), width=20)
ax.set_xlabel("Date", fontsize=16)
ax.set_ylabel("Inches", fontsize=16)
#ax.set_xticklabels([])
plt.legend(['Percipitation'], fontsize=12)
ax.get_legend().set_bbox_to_anchor((0.6, 1))
plt.xticks(rotation = 90)
plt.savefig("Resources/Percipitation.png")
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
sum_stat = prcp_scores["Precipitation"].describe()
sum_stat = pd.DataFrame(sum_stat)
sum_stat


In [ ]:
# Design a query to show how many stations are available in this dataset?
total_stations = session.query(Measurement.station).distinct().count()
print('Total number of stations is: {}'.format(total_stations))

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
active_stations = session.query(Measurement.station,
                  func.count(Measurement.station))\
.group_by(Measurement.station)\
.order_by(func.count(Measurement.station).desc()).all()
for station in active_stations:
    print(station)

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?
most_act_station = active_stations[0][0]

sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]

most_act_station_tobs = session.query(*sel).\
    filter(Measurement.station==most_act_station).all()[0]

print(f"Lowest Temperature: {most_act_station_tobs[0]}")
print(f"Average Temperature: {most_act_station_tobs[1]}")
print(f"Highest Temperature: {most_act_station_tobs[2]}")

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
high_temp = session.query(Measurement.station, Measurement.date, Measurement.tobs).\
    filter(Measurement.station == most_act_station).\
    filter(Measurement.date > date_yr_ago).order_by(Measurement.date).all()
high_temp_df = pd.DataFrame(high_temp)
high_temp_df.rename(columns={0: "Station"}, inplace=True)
high_temp_df.rename(columns={1: "Date"}, inplace=True)
high_temp_df.rename(columns={2: "tobs"}, inplace=True)
high_temp_df

In [ ]:
high_temp_df.set_index("Date")
high_temp_df.plot.hist(bins = 12)
plt.title("Temperature Observation for Station")
plt.xlabel("Temperature")
plt.ylabel("Frequency")
plt.savefig("Resources/Station.png")

## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
